In [2]:
from web3 import Web3
import os

# Initialize Web3 connection
INFURA_URL = "https://mainnet.infura.io/v3/a77f7fa2b1684b61b8de38395308ae51"

w3 = Web3(Web3.HTTPProvider(INFURA_URL))

# Contract addresses (example)
DEFAULT_COLLATERAL_ADDRESS = '0xDefaultCollateralAddress'
INPUT_TOKEN_ADDRESS = '0xUnderlyingTokenAddress'  # e.g., USDC

# Load ABIs (simplified)
with open('DefaultCollateral.abi.json') as f:
    collateral_abi = f.read()
with open('ERC20.abi.json') as f:
    erc20_abi = f.read()

# Initialize contracts
default_collateral = w3.eth.contract(address=DEFAULT_COLLATERAL_ADDRESS, abi=collateral_abi)
input_token = w3.eth.contract(address=INPUT_TOKEN_ADDRESS, abi=erc20_abi)

def deposit(amount_wei, user_address, private_key):
    # Approve token transfer
    approve_tx = input_token.functions.approve(
        DEFAULT_COLLATERAL_ADDRESS, amount_wei
    ).build_transaction({
        'from': user_address,
        'nonce': w3.eth.get_transaction_count(user_address),
    })
    signed_approve = w3.eth.account.sign_transaction(approve_tx, private_key)
    approve_hash = w3.eth.send_raw_transaction(signed_approve.rawTransaction)
    w3.eth.wait_for_transaction_receipt(approve_hash)

    # Execute deposit
    deposit_tx = default_collateral.functions.deposit(
        amount_wei, 
        user_address  # Receiver of shares
    ).build_transaction({
        'from': user_address,
        'nonce': w3.eth.get_transaction_count(user_address),
    })
    signed_deposit = w3.eth.account.sign_transaction(deposit_tx, private_key)
    deposit_hash = w3.eth.send_raw_transaction(signed_deposit.rawTransaction)
    receipt = w3.eth.wait_for_transaction_receipt(deposit_hash)
    return receipt

# Example usage
deposit_receipt = deposit(
    amount_wei=Web3.to_wei(100, 'ether'),  # Adjust decimals based on token
    user_address='0xUserAddress',
    private_key=os.getenv('PRIVATE_KEY')
)
print(f"Deposit receipt: {deposit_receipt}")

FileNotFoundError: [Errno 2] No such file or directory: 'DefaultCollateral.abi.json'

In [4]:
import json
abi=json.loads('[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"ERC1167FailedCreateClone","type":"error"},{"inputs":[],"name":"EntityNotExist","type":"error"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"entity","type":"address"}],"name":"AddEntity","type":"event"},{"inputs":[{"internalType":"address","name":"asset","type":"address"},{"internalType":"uint256","name":"initialLimit","type":"uint256"},{"internalType":"address","name":"limitIncreaser","type":"address"}],"name":"create","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"index","type":"uint256"}],"name":"entity","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"entity_","type":"address"}],"name":"isEntity","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalEntities","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]')

In [ ]:
from web3 import Web3

# Connect to Ganache (local blockchain)
ganache_url = "http://127.0.0.1:7545"
web3_ganache = Web3(Web3.HTTPProvider(ganache_url))

# Connect to Infura (Ethereum mainnet)
infura_url = "https://mainnet.infura.io/v3/a77f7fa2b1684b61b8de38395308ae51"
web3_infura = Web3(Web3.HTTPProvider(infura_url))

# DefaultCollateral contract address and ABI
contract_address = Web3.toChecksumAddress("0x1BC8FCFbE6Aa17e4A7610F51B888f34583D202Ec")
contract_abi = abi  # Replace with the actual ABI

# Initialize contract instance
contract_ganache = web3_ganache.eth.contract(address=contract_address, abi=contract_abi)
contract_infura = web3_infura.eth.contract(address=contract_address, abi=contract_abi)

# Set up account (use Ganache account for local testing)
account_address = "0xAC74e81CC49A35556E2C416accCb7BDE464927D7"
private_key = "0x7d4534266184e1bb56fd7e11b32237e2bccda04af36c29cd2daf0598a0cefbde"  # Never hardcode this in production!

# Deposit function
def deposit(amount, token_address):
    # Encode the function call
    deposit_function = contract_infura.functions.deposit(amount, token_address)
    # Build the transaction
    tx = deposit_function.buildTransaction({
        'chainId': 1,  # Ethereum mainnet
        'gas': 2000000,
        'gasPrice': web3_infura.toWei('50', 'gwei'),
        'nonce': web3_infura.eth.getTransactionCount(account_address),
    })
    # Sign the transaction
    signed_tx = web3_infura.eth.account.signTransaction(tx, private_key)
    # Send the transaction
    tx_hash = web3_infura.eth.sendRawTransaction(signed_tx.rawTransaction)
    # Wait for the transaction to be mined
    tx_receipt_deposit = web3_infura.eth.waitForTransactionReceipt(tx_hash)
    print(f"Deposit successful! Transaction hash: {tx_hash.hex()}")
    print(f"Deposit receipt: {tx_receipt_deposit}")

# Withdraw function
def withdraw(amount, token_address):
    # Encode the function call
    withdraw_function = contract_infura.functions.withdraw(amount, token_address)
    # Build the transaction
    tx = withdraw_function.buildTransaction({
        'chainId': 1,  # Ethereum mainnet
        'gas': 2000000,
        'gasPrice': web3_infura.toWei('50', 'gwei'),
        'nonce': web3_infura.eth.getTransactionCount(account_address),
    })
    # Sign the transaction
    signed_tx = web3_infura.eth.account.signTransaction(tx, private_key)
    # Send the transaction
    tx_hash = web3_infura.eth.sendRawTransaction(signed_tx.rawTransaction)
    # Wait for the transaction to be mined
    tx_receipt = web3_infura.eth.waitForTransactionReceipt(tx_hash)
    print(f"Withdrawal successful! Transaction hash: {tx_hash.hex()}")

# Example usage
if __name__ == "__main__":
    token_address = Web3.toChecksumAddress("0xYourTokenAddress")
    amount = web3_infura.toWei(1, 'ether')  # Deposit/withdraw 1 ETH (or equivalent token)
    
    # Deposit
    deposit(amount, token_address)
    
    # Withdraw
    withdraw(amount, token_address)